In [ ]:
import shutil
import hashlib
import unittest
from pathlib import Path
from datetime import datetime
import joblib
from portfawn.portfolio import BackTest
from tests.utils import get_normal_param

%matplotlib inline

## Setting the Parameters
The first step to optimize portfolios is to set the configurations such as the list of tickers, backtesting periods, and sampling and optimization paramters.

In [ ]:
## Parameters

# tickers
asset_list = ['SPY', 'IEMG', 'REET', 'BND', 'GLD']

# date
date_start = "2015-01-01"
date_end = "2020-12-31"
training_days = 8 * 5
testing_days = 5

# market
risk_free_rate = 0.0

# portfolio
portfolio_objectives = ['EWP', 'MRP', 'MVP', 'MSRP', 'BMOP']
target_return=0.2
target_sd=0.05
weight_bound=(0.02, 0.98)

# system
n_jobs = joblib.cpu_count() - 1

# backtest
backtesting_name = 'test'

# config
backtesting_config = {
    "backtesting_name": backtesting_name,
    "portfolio_fitness": portfolio_objectives,
    "tickers": tickers,
    "start_date": start_date,
    "end_date": end_date,
    "optimization_params": optimization_params,
    "sampling_params": sampling_params,
    "training_days": training_days,
    "testing_days": testing_days,
    "risk_free_rate": risk_free_rate,
    "n_jobs": n_jobs,
}

## Define Portfolios

In [ ]:
portfolio_list = []

for obj in portfolio_objectives:
    portfolio = MeanVariancePortfolio(
        objective=obj,
        target_return=target_return,
        target_sd=target_sd,
        weight_bound=weight_bound,
    )
    portfolio_list.append(portfolio)

## Backtesting

In [ ]:
portfolio_backtest = BackTest(portfolio_list)

In [ ]:
portfolio_backtest = BackTest(**backtesting_config)
portfolio_backtest.backtesting_config

In [ ]:
portfolio_backtest.run()

In [ ]:
fig, ax = portfolio_backtest.plot_returns()

In [ ]:
fig, ax = portfolio_backtest.plot_cum_returns()

In [ ]:
fig, ax = portfolio_backtest.plot_dist_returns()

In [ ]:
fig, ax = portfolio_backtest.plot_corr()

In [ ]:
fig, ax = portfolio_backtest.plot_cov()

In [ ]:
fig, ax = portfolio_backtest.plot_mean_sd()